In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy as np
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob
# from base.rl.ppo import PPO

pthfile = 'pth/ckpt.550.pth'
onnxpath = 'onnx/ckpt.550.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

inputs = torch.randn(1, 8, 240, 240)

map3 = torch.zeros(1, 240, 240)
map3[0][20][20] = 1.
inputs[0][3] = map3
inputs[0][7] = map3
x = inputs
print(inputs.size())
a = torch.squeeze(inputs,0)
print(a.size())

1.2.0
1.6.0
1.6.0
torch.Size([1, 8, 240, 240])
torch.Size([8, 240, 240])


In [5]:
class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self):
        super().__init__()
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            Flatten(),  # (G*G, )
            nn.Sigmoid(),  # added for non-negative
        )
        
    def forward(self, inputs):
        x2 = self.actor(inputs)
        return x2

In [6]:
rl_net = Global_Actor()
rl_net.actor.load_state_dict(torch.load(pthfile, map_location='cpu'),strict=False)
rl_net.eval()

x = inputs
print(x.size())
torch_out = rl_net(x)
print("pth model output:", torch_out)
print(torch_out.shape)

ort_session = onnxruntime.InferenceSession(onnxpath)

# compute ONNX Runtime output prediction
ort_outs = ort_session.run(None, {ort_session.get_inputs()[0].name: x.cpu().numpy().astype(np.float32)})

# compare ONNX Runtime and PyTorch results

print("onnx model output:", ort_outs)
print('tor_out: ', torch_out.detach().numpy().shape)

#np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
#print("Exported model has been tested with ONNXRuntime, and the result looks good!")

torch.Size([1, 8, 240, 240])
pth model output: tensor([[0.5135, 0.5228, 0.5124,  ..., 0.5090, 0.5058, 0.5083]],
       grad_fn=<SigmoidBackward>)
torch.Size([1, 57600])
onnx model output: [array([[0.46833122, 0.46837002, 0.46834514, ..., 0.46828204, 0.46827644,
        0.46824017]], dtype=float32)]
tor_out:  (1, 57600)


2022-01-05 09:21:49.595414265 [W:onnxruntime:, graph.cc:1069 Graph] Initializer 1 appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2022-01-05 09:21:49.595467624 [W:onnxruntime:, graph.cc:1069 Graph] Initializer 10 appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2022-01-05 09:21:49.595472623 [W:onnxruntime:, graph.cc:1069 Graph] Initializer 11 appears in graph inputs and will not be treated as con